# Sample Workflow with quickerstats
---
Aaron Anderson <br>
https://github.com/anderaa/quickerstats

---

In this example, we will demonstrate how to use the quickerstats package to search for and download data from the NASS Quick Stats database. Our goal will be to produce a map that shows intensity of corn production across US counties.

### 1. Installation and setup

Install the package from the current github repo, then load it.

In [81]:
library(maps)
library(maptools)
library(rgdal)

Loading required package: sp
Checking rgeos availability: TRUE
rgdal: version: 1.4-6, (SVN revision 841)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.4.2, released 2019/06/28
 Path to GDAL shared files: /Library/Frameworks/R.framework/Versions/3.6/Resources/library/rgdal/gdal
 GDAL binary built with GEOS: FALSE 
 Loaded PROJ.4 runtime: Rel. 5.2.0, September 15th, 2018, [PJ_VERSION: 520]
 Path to PROJ.4 shared files: /Library/Frameworks/R.framework/Versions/3.6/Resources/library/rgdal/proj
 Linking to sp version: 1.3-1 


In [1]:
devtools::install_github('anderaa/quickerstats', force=TRUE, build_vignettes=TRUE, ref='development')
library('quickerstats')


✔  checking for file ‘/private/var/folders/xb/3_n3p00j4_x83n2s7tpn95gm0000gn/T/RtmpLDr8Gc/remotes535b77353ede/anderaa-quickerstats-2d5de25/DESCRIPTION’ (436ms)
─  preparing ‘quickerstats’:
✔  checking DESCRIPTION meta-information
─  installing the package to build vignettes
✔  creating vignettes (2m 15.9s)
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘quickerstats_0.0.0.9003.tar.gz’
   


Now setup your NASS key. Go to https://quickstats.nass.usda.gov/api and get your key. I recommend storing your key as an environmental variable. In R studio, type `file.edit("~/.Renviron")` and add `NASS_KEY='your_nass_key'` to the file. Alternatively, open a terminal (mac/linux) and type `nano ~/.Renviron` and add the same line. Save it with ctl+o then press enter. Then exit with ctl+x.

Now load the key into the current R session:

In [2]:
key = Sys.getenv('NASS_KEY')

### 2. Searching for data 
We need to find data series that indicate the amounts of corn harvested. To do this, we use the search function.

In [59]:
r <- search_data_items(key, search_terms=c('corn', 'harvested'), exclude=c('sweet', 'silage'))
print(r)

 [1] "CORN - ACRES HARVESTED"                                                          
 [2] "CORN - OPERATIONS WITH AREA HARVESTED"                                           
 [3] "CORN, FORAGE - ACRES HARVESTED"                                                  
 [4] "CORN, GRAIN - ACRES HARVESTED"                                                   
 [5] "CORN, GRAIN - OPERATIONS WITH AREA HARVESTED"                                    
 [6] "CORN, GRAIN - PROGRESS, 5 YEAR AVG, MEASURED IN PCT HARVESTED"                   
 [7] "CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED"                               
 [8] "CORN, GRAIN - PROGRESS, PREVIOUS YEAR, MEASURED IN PCT HARVESTED"                
 [9] "CORN, GRAIN, IRRIGATED - ACRES HARVESTED"                                        
[10] "CORN, GRAIN, IRRIGATED - AREA HARVESTED, MEASURED IN PCT BY METHOD"              
[11] "CORN, GRAIN, IRRIGATED - AREA HARVESTED, MEASURED IN PCT OF OPERATIONS BY METHOD"
[12] "CORN, GRAIN, IRRIGATED - O

It looks like the first result is the one we want.

In [60]:
data_item <- r[1]
print(data_item)

[1] "CORN - ACRES HARVESTED"


Now, let's get the options associated with this data item. We need make sure we can get it for a recent year and for individual counties.

In [61]:
get_options(key, data_item)

Retrieving options...this may take a minute...
The data item is not available at the state or county level.
             There are no options.


NULL

Oops, that data item doesn't have any options associated with it, so let's try a different one.

In [62]:
data_item <- r[4]
print(data_item)

[1] "CORN, GRAIN - ACRES HARVESTED"


In [63]:
get_options(key, data_item)

Retrieving options...this may take a minute...


source_desc,year,agg_level_desc,domain_desc
<chr>,<chr>,<chr>,<chr>
CENSUS,1997,COUNTY,TOTAL
CENSUS,1997,STATE,AREA HARVESTED
CENSUS,1997,STATE,TOTAL
CENSUS,2002,COUNTY,TOTAL
CENSUS,2002,STATE,AREA HARVESTED
CENSUS,2002,STATE,TOTAL
CENSUS,2007,COUNTY,TOTAL
CENSUS,2007,STATE,AREA HARVESTED
CENSUS,2007,STATE,TOTAL


We see that census data is available at the county-level for 2017. Let's pull that data.

In [64]:
df_harvested <- get_county_data(key=key, 
                                year=2017, 
                                data_item=data_item,
                                source='CENSUS',
                                domain='TOTAL',
                                fips='all')

Parsed with column specification:
cols(
  .default = col_character(),
  asd_code = col_double(),
  region_desc = col_logical(),
  zip_5 = col_logical(),
  watershed_desc = col_logical(),
  congr_district_code = col_logical(),
  country_code = col_double(),
  year = col_double(),
  week_ending = col_logical(),
  load_time = col_datetime(format = "")
)
See spec(...) for full column specifications.


In [66]:
dim(df_harvested)
head(df_harvested)

[1] 2643   39

source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,short_desc,⋯,location_desc,year,freq_desc,begin_code,end_code,reference_period_desc,week_ending,load_time,Value,CV (%)
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<dttm>,<chr>,<chr>
CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,GRAIN,AREA HARVESTED,ACRES,"CORN, GRAIN - ACRES HARVESTED",⋯,"ALABAMA, NORTHERN VALLEY, COLBERT",2017,ANNUAL,00,00,YEAR,NA,2018-02-01,"22,148",25.7
CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,GRAIN,AREA HARVESTED,ACRES,"CORN, GRAIN - ACRES HARVESTED",⋯,"ALABAMA, NORTHERN VALLEY, FRANKLIN",2017,ANNUAL,00,00,YEAR,NA,2018-02-01,"1,101",25.7
CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,GRAIN,AREA HARVESTED,ACRES,"CORN, GRAIN - ACRES HARVESTED",⋯,"ALABAMA, NORTHERN VALLEY, LAUDERDALE",2017,ANNUAL,00,00,YEAR,NA,2018-02-01,"15,885",25.7
CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,GRAIN,AREA HARVESTED,ACRES,"CORN, GRAIN - ACRES HARVESTED",⋯,"ALABAMA, NORTHERN VALLEY, LAWRENCE",2017,ANNUAL,00,00,YEAR,NA,2018-02-01,"22,925",25.7
CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,GRAIN,AREA HARVESTED,ACRES,"CORN, GRAIN - ACRES HARVESTED",⋯,"ALABAMA, NORTHERN VALLEY, LIMESTONE",2017,ANNUAL,00,00,YEAR,NA,2018-02-01,"16,886",25.7
CENSUS,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,GRAIN,AREA HARVESTED,ACRES,"CORN, GRAIN - ACRES HARVESTED",⋯,"ALABAMA, NORTHERN VALLEY, MADISON",2017,ANNUAL,00,00,YEAR,NA,2018-02-01,"25,499",25.7


Download and extract the shapefile we need.

In [95]:
if (!file.exists('shape')) {
    dir.create('shape/')
}
download.file("https://www2.census.gov/geo/tiger/TIGER2017/COUNTY/tl_2017_us_county.zip", "shape/county_shape.zip")
unzip("shape/county_shape.zip", exdir="shape/")
file.remove("shape/county_shape.zip")

download.file("https://www2.census.gov/geo/tiger/TIGER2017//STATE/tl_2017_us_state.zip", "shape/state_shape.zip")
unzip("shape/state_shape.zip", exdir="shape/")
file.remove("shape/state_shape.zip")

[1] TRUE

[1] TRUE

In [96]:
counties <- readOGR(dsn = "shape/tl_2017_us_county.shp",
                    layer = "tl_2017_us_county")
states <- readOGR(dsn = "shape/tl_2017_us_state.shp",
                  layer = "tl_2017_us_state")

OGR data source with driver: ESRI Shapefile 
Source: "/Users/aaronanderson/NWRC/quickerstats/notebooks/shape/tl_2017_us_county.shp", layer: "tl_2017_us_county"
with 3233 features
It has 17 fields
Integer64 fields read as strings:  ALAND AWATER 
OGR data source with driver: ESRI Shapefile 
Source: "/Users/aaronanderson/NWRC/quickerstats/notebooks/shape/tl_2017_us_state.shp", layer: "tl_2017_us_state"
with 56 features
It has 14 fields
Integer64 fields read as strings:  ALAND AWATER 


In [97]:
counties_conterm <- counties[counties$STATEFP != "02" & counties$STATEFP != "15",]
states_conterm <- states[states$STATEFP != "02" & states$STATEFP != "15",]

In [98]:
conterm_proj4 <- "+proj=aea +lat_1=29.5 +lat_2=45.5 +lon_0=97.2w"
counties_conterm_albers <- spTransform(counties_conterm, CRS(conterm_proj4))
states_conterm_albers <- spTransform(states_conterm, CRS(conterm_proj4))

In [100]:
png("us-map.png", width = 10.5, height = 5, units = "in", res = 300, bg = "transparent")

# start layout
par(mar=c(0,0,0,0))
plot(0:1, 0:1, type="n", xlab="", ylab="", axes=FALSE)

# add the conterminous
par(list(new=TRUE, plt=c(0, 1, 0, 1)))

plot(counties_conterm_albers, col="#c8c8c8", border="#c8c8c8", lwd=0.1, asp=1)

ERROR: Error in plot.show(): could not find function "plot.show"


In [89]:
ogrInfo(dsn = "shape/tl_2017_us_county.shp")

Source: "/Users/aaronanderson/NWRC/quickerstats/notebooks/shape/tl_2017_us_county.shp", layer: "tl_2017_us_county"
Driver: ESRI Shapefile; number of rows: 3233 
Feature type: wkbPolygon with 2 dimensions
Extent: (-179.2311 -14.60181) - (179.8597 71.43979)
CRS: +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs  
LDID: 0 
Number of fields: 17 
       name type length  typeName
1   STATEFP    4      2    String
2  COUNTYFP    4      3    String
3  COUNTYNS    4      8    String
4     GEOID    4      5    String
5      NAME    4    100    String
6  NAMELSAD    4    100    String
7      LSAD    4      2    String
8   CLASSFP    4      2    String
9     MTFCC    4      5    String
10    CSAFP    4      3    String
11   CBSAFP    4      5    String
12 METDIVFP    4      5    String
13 FUNCSTAT    4      1    String
14    ALAND   12     14 Integer64
15   AWATER   12     14 Integer64
16 INTPTLAT    4     11    String
17 INTPTLON    4     12    String

In [79]:
library(maps)
library(maptools)
library(rgdal)


# SET DIRECTORY ####
path <- "~/Desktop/r-example/"

# read in data 
states  <- readOGR(dsn = paste(path, "census_bureau/cb_2013_us_state_20m", sep = ""),
                   layer = "cb_2013_us_state_20m")

counties  <- readOGR(dsn = paste(path, "census_bureau/cb_2013_us_county_20m", sep = ""),
                     layer = "cb_2013_us_county_20m")

# conterminous sp
counties_conterm <- counties[counties$STATEFP != "02" & counties$STATEFP != "15",]
states_conterm <- states[states$STATEFP != "02" & states$STATEFP != "15",]

conterm_proj4 <- "+proj=aea +lat_1=29.5 +lat_2=45.5 +lon_0=97.2w"
counties_conterm_albers <- spTransform(counties_conterm, CRS(conterm_proj4))
states_conterm_albers <- spTransform(states_conterm, CRS(conterm_proj4))


# USA MAP ####
png(paste(path, "us-map.png", sep=""), width = 10.5, height = 5, units = "in", res = 300, bg = "transparent")

# start layout
par(mar=c(0,0,0,0))
plot(0:1, 0:1, type="n", xlab="", ylab="", axes=FALSE)

# add the conterminous
par(list(new=TRUE, plt=c(0, 1, 0, 1)))

plot(counties_conterm_albers, col="#c8c8c8", border="#c8c8c8", lwd=0.1, asp=1)
plot(states_conterm_albers, border="white", lwd=0.6,  add=TRUE, asp=1)

# add alaska
par(list(new=TRUE, plt=c(.015, .285, .015, .260)))
plot(counties_AL_albers, col="#c8c8c8", border="#c8c8c8", lwd=0.1, asp=1)

# add hawaii
par(list(new=TRUE, plt=c(.28, .38, .015, .16)))
plot(counties_HI_albers,  col="#c8c8c8", border="#c8c8c8", lwd=0.1, asp=1)
dev.off()

ERROR: Error in delete.file("shape/us_shape.zip"): could not find function "delete.file"
